In [1]:
# load libraries
import numpy as np
import pandas as pd
from datetime import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm

In [6]:
# load data
df_train = pd.read_csv("../DERIVED/train.csv")
df_test = pd.read_csv("../DERIVED/test.csv")

C:\Users\system5\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (248,253,254,256,266,268,318,320,322) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\system5\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
print(df_train.shape)
print(df_test.shape)

(300000, 330)
(200000, 376)


In [8]:
# transfrom columns
ids = df_test['UCIC_ID'].values
df_train.drop(['UCIC_ID'], axis=1, inplace=True)
df_test.drop(['UCIC_ID'], axis=1, inplace=True)



cols = list(df_train.columns)
cols.remove('responders')
df_test = df_test[cols]



df_train = df_train.fillna(-1)
df_test = df_test.fillna(-1)



import gc
gc.collect()



for col in tqdm(cols):
    if df_train[col].dtype == 'object':
        df_train[col] = df_train[col].apply(str)
        df_test[col] = df_test[col].apply(str)

        le = LabelEncoder()
        train_vals = list(df_train[col].unique())
        test_vals = list(df_test[col].unique())
        le.fit(train_vals + test_vals)
        df_train[col] = le.transform(df_train[col])
        df_test[col] = le.transform(df_test[col])

        
        
X = np.array(df_train.drop(['responders'], axis=1))
le = LabelEncoder()
y_vals = list(df_train['responders'].unique())
le.fit(y_vals)
y = le.transform(df_train['responders'])
X_test = np.array(df_test)
del df_train, df_test; gc.collect();



X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state = 123)
del X, y; gc.collect();



d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid) 
watchlist = [d_valid]

100%|████████████████████████████████████████████████████████████████████████████████| 328/328 [00:23<00:00, 14.05it/s]


In [9]:
# light gbm parameters
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.01 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.75,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.95,
        'feature_fraction_seed': 1,
        'max_bin': 128,
        'max_depth': 10,
        'metric' : 'auc',
        'min_data_in_leaf': 10
        }

In [11]:
# light gbm model
np.random.seed(1234)
bst = lgb.train(params, d_train, num_boost_round = 1001, valid_sets = watchlist, early_stopping_rounds=50, verbose_eval=50)

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.850476
[100]	valid_0's auc: 0.852897
[150]	valid_0's auc: 0.854888
[200]	valid_0's auc: 0.857312
[250]	valid_0's auc: 0.85997
[300]	valid_0's auc: 0.862518
[350]	valid_0's auc: 0.864671
[400]	valid_0's auc: 0.866207
[450]	valid_0's auc: 0.867402
[500]	valid_0's auc: 0.868242
[550]	valid_0's auc: 0.868849
[600]	valid_0's auc: 0.869375
[650]	valid_0's auc: 0.869736
[700]	valid_0's auc: 0.87005
[750]	valid_0's auc: 0.870316
[800]	valid_0's auc: 0.870544
[850]	valid_0's auc: 0.870783
[900]	valid_0's auc: 0.870923
[950]	valid_0's auc: 0.871015
[1000]	valid_0's auc: 0.871203


In [12]:
# make predicions
p_test = bst.predict(X_test)

subm = pd.DataFrame()
subm['UCIC_ID'] = ids
subm['Responders'] = p_test
subm = subm.sort_values(['UCIC_ID'])
subm.to_csv('../SUBMISSION/submission_lightgbm_1.csv', index = False)
print('Done!')

Done!
